# Laborator 5: Deep Learning pentru Detectarea Intruziunilor

## Obiective
- Construirea unei rețele neurale MLP (Multi-Layer Perceptron)
- Înțelegerea arhitecturii și hiperparametrilor
- Antrenarea și evaluarea modelului
- (Bonus) Implementarea LSTM pentru date secvențiale

## Dataset
Vom folosi datele preprocesate din laboratoarele anterioare sau un subset din CIC-IDS2017.

## 1. Setup și Import

In [ ]:
# Instalare dependențe (doar în Colab)
# !pip install tensorflow pandas numpy scikit-learn matplotlib seaborn

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

# TensorFlow/Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LSTM, Reshape
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

# Sklearn
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                            f1_score, confusion_matrix, classification_report)

# Setări
plt.style.use('seaborn-v0_8-whitegrid')
np.random.seed(42)
tf.random.set_seed(42)

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU disponibil: {tf.config.list_physical_devices('GPU')}")

## 2. Încărcarea Datelor

In [ ]:
# Încărcăm datele
try:
    X_train = np.load('X_train.npy')
    X_test = np.load('X_test.npy')
    y_train = np.load('y_train.npy')
    y_test = np.load('y_test.npy')
    print("Date încărcate din fișiere!")
except FileNotFoundError:
    print("Creăm date sintetice...")
    from sklearn.datasets import make_classification
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import StandardScaler
    
    X, y = make_classification(
        n_samples=20000, n_features=38, n_informative=25,
        n_redundant=8, n_classes=2, weights=[0.5, 0.5],
        random_state=42
    )
    
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

print(f"\nTrain: {X_train.shape}, Test: {X_test.shape}")
print(f"Features: {X_train.shape[1]}")

In [ ]:
# Parametri
n_features = X_train.shape[1]
n_classes = 2  # Binary classification: normal vs attack

print(f"Input shape: {n_features}")
print(f"Output classes: {n_classes}")

## 3. Multi-Layer Perceptron (MLP)

### Arhitectură:
- **Input Layer**: n_features neuroni
- **Hidden Layer 1**: 128 neuroni + ReLU + Dropout
- **Hidden Layer 2**: 64 neuroni + ReLU + Dropout
- **Hidden Layer 3**: 32 neuroni + ReLU
- **Output Layer**: 1 neuron + Sigmoid (pentru binary classification)

### Componente:
- **Dense**: Layer fully-connected
- **ReLU**: Funcție de activare (Rectified Linear Unit)
- **Dropout**: Regularizare (previne overfitting)
- **BatchNormalization**: Normalizează output-ul între layere

In [ ]:
def create_mlp_model(input_dim):
    """
    Creează un model MLP pentru clasificare binară.
    
    Args:
        input_dim: Numărul de features de input
    
    Returns:
        Model Keras compilat
    """
    model = Sequential([
        # Input layer este implicit
        
        # Hidden Layer 1
        Dense(128, activation='relu', input_shape=(input_dim,)),
        BatchNormalization(),
        Dropout(0.3),
        
        # Hidden Layer 2
        Dense(64, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),
        
        # Hidden Layer 3
        Dense(32, activation='relu'),
        BatchNormalization(),
        
        # Output Layer (binary classification)
        Dense(1, activation='sigmoid')
    ])
    
    # Compilare
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Creăm modelul
mlp_model = create_mlp_model(n_features)

# Afișăm arhitectura
mlp_model.summary()

In [ ]:
# Callbacks
callbacks = [
    # Oprește antrenarea dacă validation loss nu se îmbunătățește
    EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    ),
    # Salvează cel mai bun model
    ModelCheckpoint(
        'best_mlp_model.keras',
        monitor='val_loss',
        save_best_only=True,
        verbose=1
    )
]

In [ ]:
# Antrenare
print("Antrenare MLP...")

history = mlp_model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=64,
    validation_split=0.2,
    callbacks=callbacks,
    verbose=1
)

In [ ]:
# Vizualizare curbe de învățare
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss
axes[0].plot(history.history['loss'], label='Train Loss')
axes[0].plot(history.history['val_loss'], label='Validation Loss')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training vs Validation Loss')
axes[0].legend()
axes[0].grid(True)

# Accuracy
axes[1].plot(history.history['accuracy'], label='Train Accuracy')
axes[1].plot(history.history['val_accuracy'], label='Validation Accuracy')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('Training vs Validation Accuracy')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# Evaluare pe test set
print("\nEvaluare MLP pe Test Set:")

# Predicții
y_pred_proba = mlp_model.predict(X_test)
y_pred = (y_pred_proba > 0.5).astype(int).flatten()

# Metrici
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1-Score:  {f1:.4f}")

# Matrice de confuzie
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Normal', 'Attack'],
            yticklabels=['Normal', 'Attack'])
plt.title('MLP - Confusion Matrix')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

## 4. LSTM (Long Short-Term Memory) - Bonus

LSTM este o arhitectură de rețea neurală recurentă (RNN) care poate învăța dependențe pe termen lung.

### De ce LSTM pentru securitate?
- Traficul de rețea are natură secvențială (pachete în timp)
- Atacurile pot avea pattern-uri temporale
- LSTM poate "ține minte" context din trecut

### Notă:
Pentru a folosi LSTM, trebuie să restructurăm datele în format 3D: (samples, timesteps, features)

In [ ]:
# Restructurăm datele pentru LSTM
# Simulăm secvențe de lungime 1 (fiecare sample e o "secvență" de 1 timestep)
# În practică, am grupa mai multe samples consecutive

X_train_lstm = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_lstm = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

print(f"Shape pentru LSTM: {X_train_lstm.shape}")
print(f"(samples, timesteps, features)")

In [ ]:
def create_lstm_model(input_shape):
    """
    Creează un model LSTM pentru clasificare.
    
    Args:
        input_shape: Tuple (timesteps, features)
    
    Returns:
        Model Keras compilat
    """
    model = Sequential([
        # LSTM Layer
        LSTM(64, input_shape=input_shape, return_sequences=True),
        Dropout(0.3),
        
        # Al doilea LSTM Layer
        LSTM(32, return_sequences=False),
        Dropout(0.3),
        
        # Dense layers
        Dense(32, activation='relu'),
        
        # Output
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Creăm modelul LSTM
lstm_model = create_lstm_model((1, n_features))
lstm_model.summary()

In [ ]:
# Antrenare LSTM
print("Antrenare LSTM...")

lstm_callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
]

lstm_history = lstm_model.fit(
    X_train_lstm, y_train,
    epochs=50,
    batch_size=64,
    validation_split=0.2,
    callbacks=lstm_callbacks,
    verbose=1
)

In [ ]:
# Evaluare LSTM
print("\nEvaluare LSTM pe Test Set:")

y_pred_lstm_proba = lstm_model.predict(X_test_lstm)
y_pred_lstm = (y_pred_lstm_proba > 0.5).astype(int).flatten()

lstm_accuracy = accuracy_score(y_test, y_pred_lstm)
lstm_precision = precision_score(y_test, y_pred_lstm)
lstm_recall = recall_score(y_test, y_pred_lstm)
lstm_f1 = f1_score(y_test, y_pred_lstm)

print(f"Accuracy:  {lstm_accuracy:.4f}")
print(f"Precision: {lstm_precision:.4f}")
print(f"Recall:    {lstm_recall:.4f}")
print(f"F1-Score:  {lstm_f1:.4f}")

## 5. Comparație MLP vs LSTM

In [ ]:
# Comparație rezultate
comparison = pd.DataFrame({
    'Model': ['MLP', 'LSTM'],
    'Accuracy': [accuracy, lstm_accuracy],
    'Precision': [precision, lstm_precision],
    'Recall': [recall, lstm_recall],
    'F1-Score': [f1, lstm_f1]
})

print("\nComparație MLP vs LSTM:")
print(comparison.to_string(index=False))

# Vizualizare
fig, ax = plt.subplots(figsize=(10, 6))
x = np.arange(len(comparison))
width = 0.2

metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
colors = ['steelblue', 'forestgreen', 'coral', 'purple']

for i, metric in enumerate(metrics):
    ax.bar(x + i*width, comparison[metric], width, label=metric, color=colors[i])

ax.set_xlabel('Model')
ax.set_ylabel('Score')
ax.set_title('Comparație Deep Learning Models')
ax.set_xticks(x + width * 1.5)
ax.set_xticklabels(comparison['Model'])
ax.legend()
ax.set_ylim([0.8, 1.0])

plt.tight_layout()
plt.show()

## 6. Salvarea Modelelor

In [ ]:
# Salvăm modelele
mlp_model.save('mlp_model.keras')
lstm_model.save('lstm_model.keras')

# Salvăm rezultatele
comparison.to_csv('deep_learning_results.csv', index=False)

# Salvăm istoricul antrenării
with open('mlp_history.pkl', 'wb') as f:
    pickle.dump(history.history, f)

print("Modele și rezultate salvate!")
print("  - mlp_model.keras")
print("  - lstm_model.keras")
print("  - deep_learning_results.csv")
print("  - mlp_history.pkl")

## 7. Rezumat

### Ce am învățat:
1. **MLP** - Rețea feedforward cu multiple layere dense
   - Bun pentru date tabelare
   - Rapid de antrenat
   
2. **LSTM** - Rețea recurentă cu memorie
   - Bun pentru date secvențiale
   - Mai complex, mai lent

### Hiperparametri importanți:
- `learning_rate`: Viteza de învățare (0.001 e un start bun)
- `epochs`: Numărul de iterații
- `batch_size`: Câte samples per update
- `dropout`: Rata de dropout pentru regularizare

### Tehnici anti-overfitting:
- Dropout
- BatchNormalization
- EarlyStopping

### Următorul pas:
**Laborator 6** - Evaluare comparativă și vizualizare rezultate